### Realiza conexão com site <a href='https://www.cachacaexpress.com.br/'>Cachaça Express </a> e obtem informações relevantes sobre suas cachaças.

In [1]:
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

#### Obtendo a página inicial do <i>site</i>

In [2]:
print('[+] Iniciando...!')

endereco = 'https://www.cachacaexpress.com.br/cachaca.html?dir=desc&limit=all'

# Realizando conexão com site e lendo a página html
html = requests.get(endereco).text

# Criando um objeto BeautifulSoup com o html da página
bsObj = BeautifulSoup(html, 'html.parser')
bsObj.title.text

[+] Iniciando...!


'As melhores cachaças artesanais de Minas e do Brasil - Cachaça Express'

#### Obtendo <i>links</i> para página de informação de cada cachaça

In [3]:
# Obtendo endereço da página com informações de cada cachaça
cachaca_list = bsObj.find_all('h2', {'class': 'product-name'})

# Obtém a lista url de informações de todas cachaças
url_list = [url.a.attrs['href'] for url in cachaca_list]

print('Total de cachaça: ', len(cachaca_list))
url_list[:10]

Total de cachaça:  335


['https://www.cachacaexpress.com.br/cachaca/cachaca-vale-verde-3-anos-700ml.html',
 'https://www.cachacaexpress.com.br/cachaca/cachaca-vale-verde-3-anos-50ml.html',
 'https://www.cachacaexpress.com.br/cachaca/cachaca-minha-deusa-700ml.html',
 'https://www.cachacaexpress.com.br/cachaca/cachaca-vale-verde-12-anos-700ml.html',
 'https://www.cachacaexpress.com.br/cachaca/cachaca-coluninha-50-ml.html',
 'https://www.cachacaexpress.com.br/cachaca/cachaca-prosa-700.html',
 'https://www.cachacaexpress.com.br/cachaca/cachaca-boazinha-670ml-ouro.html',
 'https://www.cachacaexpress.com.br/cachaca/cachaca-boazinha-50ml-ouro.html',
 'https://www.cachacaexpress.com.br/cachaca/cachaca-lua-nova-700ml-ouro.html',
 'https://www.cachacaexpress.com.br/cachaca/cachaca-canarinha-600ml.html']

#### Conectando a página de cada produto e obtendo informações

In [4]:
def get_avaliacoes(indice, bs_cachaca):    
    '''
    Obtém as avaliações de uma determinada cachaça, caso exista. Recebe um
    objeto BeautifulSoup da página da cachaça e o índice
    referente a cachaça.
    '''
    
    avaliacao_list = []
    avaliacoes = bs_cachaca.select('div[id="product-customer-reviews"] li')
    
    for x in avaliacoes:

        avaliacao = x.find_all('p')
        
        # Se não tem nenhuma avaliação, retorne
        if len(avaliacao) == 0:
            return []
            
        # Obtem satisfação do cliente
        satisfacao = x.find_all('div', style=True)
        
        if len(satisfacao) != 0:
            satisfacao = satisfacao[0]['style'].split()[1].replace(';', '')
        else:
            satisfacao = None
        
        # Armazena o indice da cachaça, a avaliação e data da avaliação
        avaliacao_list.append(
            [
                indice, satisfacao,
                avaliacao[0].text.replace('\"', ''), avaliacao[1].span.text.split()[1]
            ]
        )
        
    return avaliacao_list
    
    
avaliacao_list = []
info_list = []
total = len(url_list)

for indice, url in enumerate(url_list):
    
    # Realizando conexão com a url, obtendo html e criando um objeto BeautifulSoup.
    html = requests.get(url).text
    bs_cachaca = BeautifulSoup(html, 'html.parser')

    # Obtendo informações da cachaça
    info = [info.text.strip() for info in bs_cachaca.tbody('td')]
    
    # Obtendo preço
    info.append(
        bs_cachaca.select('div[class="product-shop-info"] span[class="price"]')[0].text
    )    
    
    info_list.append(info)
    avaliacao_list += get_avaliacoes(indice, bs_cachaca)
    
    print('[+] {0} <--> {1}\r'.format(indice +1, total), end='')

#### Criando <i>DataFrame</i> com informações das cachaças

In [5]:
# Criando DataFrame com as informações das cachaças
labels = [
    'descricao', 'estado', 'envelhecimento', 'alcool', 'marca',
    'madeira', 'grupo', 'madeira', 'tipo', 'volume', 'preco'
]
df_cachaca = pd.DataFrame(info_list, columns=labels)
df_cachaca.head()

,descricao,estado,envelhecimento,alcool,marca,madeira,grupo,madeira,tipo,volume,preco
0,Envelhecida em barris de carvalho por três ano...,Minas Gerais,3 anos,40% Vol.,Vale Verde,Carvalho,Cachaça,Carvalho,Não,700 ml,"R$79,90"
1,A cachaça Vale Verde é produzida em um parque ...,Minas Gerais,3 anos,40% Vol.,Vale Verde,Carvalho,Cachaça,Carvalho,Não,50 ml,"R$9,95"
2,A Cachaça Minha Deusa é produzida pela Vale Ve...,Minas Gerais,Sem envelhecimento,40% Vol.,Vale Verde,Não Envelhcida,Cachaça,Não Envelhcida,Não,700 ml,"R$34,95"
3,"A cachaça Vale Verde 12 anos, produzida em um ...",Minas Gerais,12 anos,40% Vol.,Vale Verde,Carvalho,Cachaça,Carvalho,Não,700 ml,"R$594,95"
4,A cachaça Coluninha é uma tradicional cachaça ...,Minas Gerais,Não,40% Vol.,Coluninha,Não Envelhcida,Cachaça,Não Envelhcida,Não,50 ml,"R$8,50"


#### Criando <i>DataFrame</i> com avaliações das cachaças

In [6]:
# Criando DataFrame com as informações das avaliações
labels = ['id_cachaca', 'satisfacao', 'avaliacao', 'data_avaliacao']
df_avaliacao = pd.DataFrame(avaliacao_list, columns=labels)
df_avaliacao.head()

,id_cachaca,satisfacao,avaliacao,data_avaliacao
0,0,100%,"Excelente, envelhecida no carvalho, premiadíss...",25/11/2012
1,0,100%,É outro nível de cachaça. Muito boa. Não é par...,05/11/2012
2,0,100%,E beber com estilo,25/01/2012
3,0,100%,Melhor cachaça que já tomei!,26/08/2011
4,0,100%,A cachaça Vale Verde é a melhor cachaça do Bra...,09/08/2011


#### Salvando os <i>DataFrame</i> para arquivo .csv

In [7]:
df_cachaca.to_csv('dataset/cachaca.csv', )
df_avaliacao.to_csv('dataset/avaliacao.csv')

### Rápida análise dos dados

In [8]:
# lendo arquivos
df_aguardente = pd.read_csv('dataset/cachaca.csv')
df_ava = pd.read_csv('dataset/avaliacao.csv')

In [9]:
# Total de avaliçãoes
print('Total de cachaças: ', df_aguardente.count().marca)
print('Total de avaliações: ', df_ava.groupby(['id_cachaca']).count().shape[0])

Total de cachaças:  335
Total de avaliações:  13
